<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv3/blob/main/virustotal_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://drive.google.com/file/d/103hYqdHD6mjAUVqMAECgZEZ7U7YmSCP9/view?usp=sharing
import gdown
download_link = 'https://drive.google.com/uc?id=103hYqdHD6mjAUVqMAECgZEZ7U7YmSCP9'
output_filepath = '/content/'
gdown.download(download_link, output_filepath)

Downloading...
From: https://drive.google.com/uc?id=103hYqdHD6mjAUVqMAECgZEZ7U7YmSCP9
To: /content/drebin_drift.parquet.zip
100%|██████████| 16.6M/16.6M [00:00<00:00, 143MB/s]


'/content/drebin_drift.parquet.zip'

In [2]:
import pandas as pd
df = pd.read_parquet("/content/drebin_drift.parquet.zip")
df.head()

,sha256,submission_date,api_call,permission,url,provider,feature,intent,activity,call,service_receiver,real_permission,label
0,a44a4856e27a0f8448a12f445b46d9430b93fd15414b04...,2009-06-08 02:21:25,android/media/MediaPlayer->start android/app/A...,android.permission.VIBRATE,,,android.hardware.touchscreen,android.intent.action.MAIN android.intent.cate...,.FlAndroidApp FlAndroidApp,getSystemService,,android.permission.INTERNET android.permission...,0
1,05ffa80882edba811018e95e0284b1b77f36a5e0a85236...,2009-06-08 02:25:49,android/media/MediaPlayer->stop android/os/Vib...,android.permission.VIBRATE android.permission....,,,android.hardware.touchscreen android.hardware....,android.intent.action.MAIN android.intent.cate...,.GloftMPL2,getSystemService,,android.permission.MODIFY_AUDIO_SETTINGS andro...,0
2,788411a922a37bdd731902272a2e22530bea75d6d44bc9...,2009-06-08 02:29:07,android/media/MediaPlayer->start android/app/A...,,,,android.hardware.touchscreen,android.intent.action.MAIN android.intent.cate...,.FlAndroidApp FlAndroidApp,getSystemService,,android.permission.INTERNET android.permission...,0
3,2333f5dcc54a3332dfe0414eb97cddddc8307ee829c47b...,2009-06-08 02:29:15,android/media/MediaPlayer->start android/os/Vi...,android.permission.VIBRATE android.permission....,,,android.hardware.touchscreen android.hardware....,android.intent.action.MAIN android.intent.cate...,.GloftORTR,getSystemService,,android.permission.MODIFY_AUDIO_SETTINGS andro...,0
4,0b4094ec8c1c6a7e03881e4c49f1bbd8510e0f3c818cca...,2009-07-18 23:29:00,java/lang/Runtime->exec,android.permission.ACCESS_SUPERUSER,,,android.hardware.touchscreen,android.intent.action.MAIN android.intent.cate...,.ScreenshotActivity,Execution of external commands,,android.permission.READ_LOGS,0


In [26]:
import requests

# Replace with your own VirusTotal API key
api_key = "56daf7438dd2529bc555572bfccf3e15bda412c415c5a8510918d092fbe2cddf"

# Replace '{id}' with SHA-256, SHA-1 or MD5 identifying the file
file_id = 'd7e1d0fb43c7d1d86d14a0e036539f5790656a8aa4842c8325c2b35790bf597f'

# Define the URL for the VirusTotal API
url = f'https://www.virustotal.com/api/v3/files/{file_id}'

# Set up the headers with your API key
headers = {
    'x-apikey': api_key
}

# Send a GET request to retrieve information about the file
response = requests.get(url, headers=headers)

# Access the 'last_analysis_results' field
analysis_results = response.json().get('data', {}).get('attributes', {}).get('last_analysis_results', {})

# Initialize a count for undetected results
detected_count = 0
undetected_count = 0
unsupported_count = 0

# Check each engine's result and count undetected results
for engine, result in analysis_results.items():
    if result['category'] == 'undetected':
        undetected_count += 1
    if result['category'] == 'malicious':
        detected_count += 1
    if result['category'] == "type-unsupported" :
        unsupported_count += 1


print('Probability of malware : ', detected_count/(detected_count + undetected_count + 0.0001))
print("Number of undetected results:", undetected_count)
print("Number of detected results:", detected_count)
print("Number of unsupported results:", unsupported_count)

Probability of malware :  0.0
Number of undetected results: 0
Number of detected results: 0
Number of unsupported results: 0


In [13]:
analysis_results

{}

In [7]:
def n_malicious(file_id):
  # Replace with your own VirusTotal API key
  api_key = "56daf7438dd2529bc555572bfccf3e15bda412c415c5a8510918d092fbe2cddf"

  # Define the URL for the VirusTotal API
  url = f'https://www.virustotal.com/api/v3/files/{file_id}'

  # Set up the headers with your API key
  headers = {
      'x-apikey': api_key
  }

  # Send a GET request to retrieve information about the file
  response = requests.get(url, headers=headers)

  # Access the 'last_analysis_results' field
  analysis_results = response.json().get('data', {}).get('attributes', {}).get('last_analysis_results', {})

  # Initialize a count for undetected results
  detected_count = 0

  # Check each engine's result and count undetected results
  for engine, result in analysis_results.items():
      if result['category'] == 'malicious':
          detected_count += 1

  return detected_count

In [27]:
import pandas as pd
import requests

# Replace with your own VirusTotal API key
api_key = "56daf7438dd2529bc555572bfccf3e15bda412c415c5a8510918d092fbe2cddf"

def is_malicious(file_id):
    """
    Checks if a file is malicious using VirusTotal API.

    Args:
        file_id: The SHA256 hash of the file.

    Returns:
        True if the file is considered malicious, False otherwise.
    """

    url = f"https://www.virustotal.com/api/v3/files/{file_id}"
    headers = {"x-apikey": api_key}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error if API request fails

        analysis_results = response.json().get("data", {}).get("attributes", {}).get("last_analysis_results", {})

        for engine, result in analysis_results.items():
            if result["category"] == "malicious":
                return True

        return False

    except requests.exceptions.RequestException as e:
        print(f"Error fetching API data for {file_id}: {e}")
        return False  # Assume not malicious on error


In [29]:
is_malicious('d7e1d0fb43c7d1d86d14a0e036539f5790656a8aa4842c8325c2b35790bf597f')

Error fetching API data for d7e1d0fb43c7d1d86d14a0e036539f5790656a8aa4842c8325c2b35790bf597f: 429 Client Error: Too Many Requests for url: https://www.virustotal.com/api/v3/files/d7e1d0fb43c7d1d86d14a0e036539f5790656a8aa4842c8325c2b35790bf597f


False

In [8]:
df_benign = df[df['label']==0]
df_malicious = df[df['label']==1]

In [ ]:
from tqdm import tqdm

wrong_lables = []
for file_id in tqdm(df_benign['sha256']):
  count = n_malicious(file_id)
  if count > 5 :
      wrong_lables.append(file_id)
      print(file_id)
      print(count)


  7%|▋         | 8247/123453 [27:26<9:06:56,  3.51it/s]

d7e1d0fb43c7d1d86d14a0e036539f5790656a8aa4842c8325c2b35790bf597f
24


  7%|▋         | 8256/123453 [27:30<11:20:31,  2.82it/s]

2429d1f05204ff343d469ebc99b0769e32cba4c50fbf8ec48c5c612a78391567
20


  7%|▋         | 8271/123453 [27:42<19:30:48,  1.64it/s]

...

  8%|▊         | 9361/123453 [32:33<11:15:08,  2.82it/s]

b63d9b29789de547072792543faff352d3bb9af7dcbb99b47e5bfb37970972c3
19


  8%|▊         | 9362/123453 [32:34<10:32:55,  3.00it/s]

2257a9d845a3ec120f6ed9de265e427628f592f3b1381753a9d2af71d0ee5588
18


 51%|█████     | 63093/123453 [3:56:52<3:25:48,  4.89it/s]